### Importing and setting up

In [1]:
import time
from threading import Thread
import logging
import os
from datetime import datetime
import re

# import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

In [3]:
all_links = set()

In [4]:
chrome_options = Options()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--disable-session-crashed-bubble")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--suppress-message-center-popups")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("disable-infobars")

chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--window-size=1920x1080")

# chrome_options.add_argument("--no-first-run")
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
# chrome_options.add_experimental_option('useAutomationExtension', False)
# chrome_options.add_experimental_option("excludeSwitches", ["enable-logging", "test-type"])

### Essential Function in filtering

In [5]:
def clearing_click_on_server():
    servers_clear_box_xpath = '/html/body/app-root/div/div[2]/servers/servers-container/div/aside/app-server-filter/div/div[1]/div/a[1]'
    servers_clear_box = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, servers_clear_box_xpath))
    )
    servers_clear_box.click()

In [6]:
def open_filter_dropdown():
    filter_xpath = '/html/body/app-root/div/div[2]/servers/servers-container/div/aside/app-server-filter/div/section[2]/div/div[1]'
    filters = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, filter_xpath))
    )
    filters.click()
    time.sleep(10)

### 3 Core filtering functions

In [7]:
def custom_server_search(server_name_to_search):
    search_bar_xpath = '//*[@id="searchBox"]'
    search_bar = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, search_bar_xpath))
    )
    search_bar.click()
    search_bar.send_keys(f'{server_name_to_search}', Keys.RETURN)
    
    clearing_click_on_server()    
    

In [8]:
def filter_languages(language_name):
    open_filter_dropdown()
    languages_xpath = '/html/body/app-root/div/div[2]/servers/servers-container/div/aside/app-server-filter/div/section[2]/div/div[3]/app-server-tag-filter/div/div[1]'
    language_filters = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, languages_xpath))
    )
    language_containing_xpath = f".//*[contains(text(), '{language_name}')]"
    language_clicks = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, language_containing_xpath))
    )
    time.sleep(4)
    language_clicks.click()
    clearing_click_on_server()    

In [9]:
def filter_tags(tag_name):
    open_filter_dropdown()
    tags_xpath = '/html/body/app-root/div/div[2]/servers/servers-container/div/aside/app-server-filter/div/section[2]/div/div[3]/app-server-tag-filter/div/div[2]/ul'    
    tag_filters = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, tags_xpath))
    )
    tag_click_action = ActionChains(driver)
    tag_click_action.move_to_element(tag_filters).scroll_by_amount(0, 500).perform()
    time.sleep(4)
    tag_containing_xpath = f".//*[contains(text(), '{tag_name}')]"
    print(tag_containing_xpath)
    tag_clicks = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.XPATH, tag_containing_xpath))
    )
    tag_click_action = ActionChains(driver)
    tag_click_action.move_to_element(tag_clicks).click().perform()
    tag_clicks.click()
    clearing_click_on_server() 

### Main Script for scraping servers

In [10]:
def scrape_server_links():
    print("Please write an estimate number of servers to scrape.")
    estimated_servers = int(input("A number only will work: "))
#     mind to keep on same page on all filtering
#     driver = webdriver.Chrome(service=Service(r"chromedriver.exe"), options=chrome_options)
#     driver.get('https://servers.fivem.net/servers')

    container_list_xpath = "//servers-list[1]//div[contains(@class, 'cdk-virtual-scroll-content-wrapper')]"

    container_list = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, container_list_xpath))
    )

    i = 0
    # for i in range(100):
    while len(all_links) < estimated_servers:
        i = i+1
        servers_loaded = WebDriverWait(container_list, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, './/servers-list-item//img'))
        )
        servers_loaded_no = len(servers_loaded) - 1
        print(f"Iteration {i}: Scraped a total of {servers_loaded_no} loaded servers.")
    #     print("The last loaded server text is: ", servers_loaded[servers_loaded_no].text)

        iteration_links = []

        for (i, images_loaded_server) in enumerate(servers_loaded):
            link_pict = images_loaded_server.get_attribute('src')
            server_id = link_pict.split('/')[-2]
    #         print(server_id)
            iteration_links.append(server_id)
    #         print(iteration_links)

        iteration_links = set(iteration_links)
        print(iteration_links)

        with open('Backup Links.txt', 'w') as f:
            for link in iteration_links:
                f.write(link)
                f.write('\n')

        print("These links are obtained in this iteration: ")
    #     print(iteration_links)
        print(len(iteration_links))
        all_links.update(iteration_links)
        print(len(all_links))

        time.sleep(5)
        last_loaded_server = ActionChains(driver)
        last_loaded_server.scroll_to_element(servers_loaded[servers_loaded_no]).perform()
        print("Scrolled to last server.")

    with open('All Links.txt', 'w') as f:
        for link in all_links:
            f.write(link)
            f.write('\n')

    print("Concluded with scraping!!!")
    print(f"Total {len(all_links)} servers scraped")

In [11]:
# driver = webdriver.Chrome(service=Service(r"chromedriver.exe"), options=chrome_options)
# driver.get('https://servers.fivem.net/servers')
# scrape_server_links(driver)

In [12]:
driver = webdriver.Chrome(service=Service(r"chromedriver.exe"), options=chrome_options)
driver.get('https://servers.fivem.net/servers')

In [13]:
def setup_server_listings():
    print("Starting with scraping the results...")
    print("Please interact with following inputs to filter your servers.\n")
    
    print("Do you want to search by any custom server names?")
    print("""
    Note: you can search only one server name at this time,
    if you want to search multiple server names, you have to run this script multiple times.
    """)
    
    try:
        custom_bol = input("If yes, type 'y', otherwise skip custom server name search by entering no. ")
        if custom_bol == 'y':
            print("Write custom server name below.")
            custom_bol = input("Name: ")
            print("Going to search for: ")
            print(custom_bol.strip())
            custom_server_search(custom_bol.strip())        
    except Exception as Ex:
        print("There is an error in filtering by custom server name.")

    
    print("Do you want to filter by any language?")
    lang_bol = input("If yes, type 'y', otherwise skip language filtering by entering no. ")
    if lang_bol == 'y':
        print("Write comma separated names of languages below: ")
        lang_list = input("Languages: ")
        langs = lang_list.split(',')
        langs = [s.strip() for s in langs]
        
        print("Going to filter by: ")
        print(langs)
        for i in langs:
            try:                
                print(i)
                filter_languages(i)  
            except Exception as Ex:
                print(f"There is an error in filtering by language {i}. ")
                print(Ex)
    
    print("Do you want to filter by any tags?")
    tag_bol = input("If yes, type 'y', otherwise skip tags filtering by entering no. ")
    if tag_bol == 'y':
        print("Write comma separated tags below: ")
        tag_bol = input("Tags: ")
        tags = tag_bol.split(',')
        
        tags = [t.strip() for t in tags]
        print("Going to filter by: ")
        print(tags)
        
        for i in tags:
            try:                
                print(i)
                filter_tags(i)  
            except Exception as Ex:
                print(f"There is an error in filtering by tag {i}")
                print(Ex)
    
    try:
#         clearing_click_on_server()
        scrape_server_links() 
    except Exception as Ex:
        print("An error occured scraping a list of servers.")
        print(Ex)

    print("The script ended running successfully.")
    print("Thank you for working with me!")
    print("")

In [14]:
setup_server_listings()

Starting with scraping the results...
Please interact with following inputs to filter your servers.

Do you want to search by any custom server names?

    Note: you can search only one server name at this time,
    if you want to search multiple server names, you have to run this script multiple times.
    
If yes, type 'y', otherwise skip custom server name search by entering no. 
Do you want to filter by any language?
If yes, type 'y', otherwise skip language filtering by entering no. 
Do you want to filter by any tags?
If yes, type 'y', otherwise skip tags filtering by entering no. 
Please write an estimate number of servers to scrape.
A number only will work: 6000
Iteration 1: Scraped a total of 41 loaded servers.
{'pk54ay', 'br4qab', 'r3ma3p', '99vvlm', '6b5ro7', 'yk9l85', 'aqqrk5', 'wmpmb9', 'dby6gj', '5rbl4a', 'xdbm3m', 'evgyp3', '4epke8', '8g3ep5', 'zpxbx9', 'yjor85', 'bebjjp', '5rmd47', 'jklk7k', 'e3xz9b', 'p555oy', 'x43r9m', 'x63aoe', 'bgx6yb', 'q8538p', 'zgk8op', '4pyg5q', 

Scrolled to last server.
Iteration 84: Scraped a total of 74 loaded servers.
{'yjgojy', '8kgomm', 'xdvq3g', 'ooxkkx', 'p48mb8', 'ej46gb', '755ole', 'evgrrd', 'gjeaoz', 'oyl857', 'zpxjvp', 'mkvbqa', 'dpl5l5', '3kj9zb', 'y6kqk9', 'z69qgy', 'pprvvy', '5q8v96', 'data:image', '4k47q5', 'yjy469', 'qom3lz', 'devmyj', 'vyqkze', 'jvqexp', 'deyzxq', 'lm6b54', '7xokzd', 'myqemv', 'oygp7y', 'gjjpmz', 'ogmr5x', 'z98oby', 'b8zaxd', 'lz4zo4', 'gogj8q', 'oo8rrv', 'z3b49y', '7kydev', '7drdvr', '4agj4q', 'vmz58k', '9gpprm', 'r8b5y8', 'gxmjlq', 'j55o9p', 'mk9v3v', 'zzbmop', 'je6jam', '5e54k7', 'vdaalq', 'kyvkgr', '438zo9', '9qv34y', '6do5aj', 'o3xllx', '4remv8', '8gzvb3', '834q85', '6zay36', 'x69xdr', '5qdara', '6vm8z8', 'aqb9v9', 'l66blo', 'mo9kod', 'momkav', '3kgv6b', 'poa758', 'x3o5eg', '9lxx6e', 'jkk7jp', '5eoypa', 'pyxy35'}
These links are obtained in this iteration: 
74
301
Scrolled to last server.
Iteration 75: Scraped a total of 79 loaded servers.
{'oyrqxr', 'o46amx', 'b8eko8', 'qom3lz', 'yjy469'

Scrolled to last server.
Iteration 74: Scraped a total of 83 loaded servers.
{'46gkao', 'kkgjgd', 'b8eko8', 'a5doj9', 'x5b4yx', '433l65', 'objpqy', 'rd4r9j', '8g3q64', 'karm4a', 'b9dry8', 'bvljql', '4kpmr9', 'de5dzy', 'bvkvzp', '9b5yre', 'dpzvpq', 'gboalm', 'jggbal', 'ajmv79', 'ez4xo3', '8g4zkb', 'bbdee4', 'vekdkx', '3k5loy', 'apyz69', 'x3o87m', '38ea8o', 'e4xvym', 'z67lgp', 'jklppp', 'kxyomv', '9rzxdm', 'xda4gr', '69pa4j', '5blaaz', 'prg88m', '8j5mjv', 'mv8ebv', 'e4lrd3', 'kykybr', 'ggv7x9', 'bdmdx4', '59p7qa', 'data:image', 'j69x6m', 'br9d5p', '438xvq', 'l6bokv', 'zgedoy', 'j4gokk', 'pqqvj8', 'ye9d6y', '3l98ko', 'gblz4q', 'ykvjey', 'pkmqqy', '8j9m9b', '8lkvbv', '5zad5r', 'x6zdxr', 'j6rv5a', 'ydpmy9', 'rlqpvp', '99yvdy', 'mbzev9', 'kk6jov', 'lpdpk4', '9glgxy', 'x3lqyx', 'zgveb5', 'gz855z', 'mkyv9d', 'zqje39', '948a5m', '8z96y3', 'pyp5vy', '66ly86', '5e8ga7', '4e7gl5', '389drr', 'md4bl7'}
These links are obtained in this iteration: 
82
556
Scrolled to last server.
Iteration 84: Scraped

Scrolled to last server.
Iteration 84: Scraped a total of 79 loaded servers.
{'b38gd8', 'jokyx4', 'jby9vm', 'kkgjgd', 'de85yq', '6qmz34', '9qolye', 'qpv6qp', '6zxq47', 'qqk3pz', '6qjag4', 'a4g4o3', 'rbxeag', '5z8zbz', '9gad7k', 'o3or9v', 'q3l6d4', '6d7yq6', 'r3j67g', '3959by', 'byrrrp', 'kl8696', '83bmm5', 'apyz69', '5e866z', 'yrzx4k', '94oyle', 'npqdg5', '788eer', 'xg55kr', 'yqk3am', 'yjagdj', 'xlq7bm', 'mj5ee9', 'ryzzvp', 'z3ml35', '5edgxa', '7d6v3a', 'a6ev6e', '5eb4ka', 'y9y5yj', 'lzeb9j', 'data:image', 'laxbbo', 'lxjde7', '45xl8q', 'go4oro', '9dax3y', '4eelro', 'poo4xa', 'yjop4j', 'x94jgm', 'ejazga', 'd54g73', 'q2llv2', '7j8z4d', 'j5keok', 'zzj6b4', '78zd3a', 'j6ovgp', '43qgg8', '745krv', 'xdrzgg', 'y66r4m', 'pkpj5y', 'zzo655', 'zgveb5', 'ry6jvg', 'dv9v85', 'yrx4vy', 'jgvmom', 'mkrlz9', 'byd67p', 'mo3j7q', '8b3alm', 'kaom5d', '9az5ee', 'gzyzgz'}
These links are obtained in this iteration: 
78
796
Scrolled to last server.
Iteration 80: Scraped a total of 83 loaded servers.
{'jokyx4'

Scrolled to last server.
Iteration 100: Scraped a total of 94 loaded servers.
{'kkgjgd', 'mb58j7', '9qolye', 'pxbyoa', '9gyymk', '6377dj', '834j4b', 'yj4oxm', '8gb95v', 'b8b75l', 'dxzq6j', '7lg5rb', '2vvomq', 'e6drv3', 'ogm4yx', 'ezg7em', 'qzo4z9', 'g5a8ax', 'qz48dv', '4edq3q', 'e4488a', 'vmjb45', 'abvp95', 'oy7mpx', '5bgj66', '9gad7k', 'o3or9v', 'wmev57', 'galpex', 'v57pqk', 'k4zj86', 'byrrrp', 'poyja7', 'apyz69', 'q33xb4', 'pkde38', 'qpvzl6', '9rgbbk', 'dz87jy', '8qjkgb', 'yjagdj', 'pygzmm', 'o4j4dj', '8g8774', '4383mq', 'e4q5lb', 'zmgyly', 'z645kd', 'j6pygk', 'p8x9p7', 'l6keoo', 'gk5mgz', 'xglzmm', '83op83', 'zqlaxd', 'data:image', 'p8zbm7', 'zgxeep', '6qe888', 'yalq5k', 'a6r8o3', 'yxyo9m', '9xk6qk', 'x94jgm', '836v7m', 'goamlz', 'p5m6v8', 'olk467', 'bdmgb4', 'kk8zva', 'zzj6b4', 'ajqogk', 'vlxk3x', 'edjk43', 'zpoxl9', 'j6ovgp', '5z95ga', 'yj3ojk', 'm649l7', 'jbmaek', 'qy574v', 'e5ebpb', 'm4v7dq', 'zzo655', 'e4oqg3', 'e4bdqp', 'ye9jrm', 'la9z5j', '83xrg4', 'vooayq', 'jkooxm', 'e4p9kp

...

Scrolled to last server.
Iteration 92: Scraped a total of 82 loaded servers.
{'v5bm4q', 'ogmqjr', '6mgm5j', 'gk5z6o', 'xlx4eg', 'evylqd', 'ema5aa', '3lqjdo', '9ldbry', 'lpke45', 'xpzeme', 'xdl6dm', '8jj7kb', 'jpap8p', 'lzebx4', '58em47', 'za4p45', 'e8o4ma', '6zgkk8', 'gzov89', 'ye99zj', '56pyoa', 'vj56yx', 'rvgpvg', 'vp8kq5', '9eomeb', '8balgm', 'xk6b9x', '664dq6', '5r5p3d', '8ze7b4', 'vegple', '5rb6qz', 'qo7jy4', 'dbrx3d', 'o4yppr', 'brdy9p', 'zgbda4', 'ke34d6', 'gkrz7z', 'md369v', 'zgkxvp', 'dbpxyy', '4qk45o', 'b6ra7d', 'qyek86', 'rkjop7', 'j4jzbl', 'e4bx4p', 'yed3aj', 'brdp7d', 'keqo7v', '66m6d8', 'ab8xrz', 'data:image', '4q77o5', 'dv9pyd', '83xppv', 'vjjb8r', 'py7bq7', 'vyqjdk', 'qyaabp', '9g7qpb', '3abzey', 'bgqgx4', 'qkp7m6', 'veg9ye', 'j3m6l4', 'y6b58m', 'e4mrqb', 'gb3qeo', 'kx4lbv', 'l693v4', 'rvmo7j', '83lrk5', 'la7vaj', '6q9rk7', '785y6e', 'k4voja', 'agzm83', '3854m8', 'xgajde'}
These links are obtained in this iteration: 
82
5111
Scrolled to last server.
Iteration 83: Scrape

Scrolled to last server.
Iteration 86: Scraped a total of 78 loaded servers.
{'rd9j7j', 'pk98ey', 'x3lpxg', '4g4do9', 'xdy3dg', 'jkaljl', 'l6rdo5', 'k49ydr', 'm3oz4d', 'dbv3jq', 'x4by8g', 'goavxo', 'abp8vk', 'dbm69d', 'yekqzy', 'rkpk5x', 'yygvgm', '56yry7', 'eme9qb', 'd8dz45', '76bokr', 'a534d3', 'l6vey7', 'j64pym', '26aard', 'pogabm', 'rkoymg', 'ye8q89', 'xdv3ge', 'bqemr8', 'qyxxq4', 'goq4kq', '6zp33d', 'a6qqez', 'zpd985', 'bgleep', '9bog8a', '86l6em', 'yjq8by', '86mxob', 'e4ebmd', '6za9e7', 'gk4avq', 'ag4bbz', 'ogjoa7', 'pkzpx8', '6mgk88', 'data:image', 'verk9k', 'qo4b84', 'kev3vv', 'zzovl5', 'xdmkox', 'ye986k', 'dvg873', '5q9z77', '9ol4ob', 'pggoe5', 'kkj9yd', 'gk3o9o', 'gegexz', 'jprk4p', 'qpv3gp', 'vyby9q', 'abo3ee', 'mkll8d', 'dvg3oy', 'z347y9', '6bavr7', '8jp935', 'bbargp', 'kggp9v', 'e5xkd3', '66vgd8', '8xe5r5', 'ooe4gv', '2l53v5'}
These links are obtained in this iteration: 
77
5338
Scrolled to last server.
Iteration 79: Scraped a total of 81 loaded servers.
{'pk98ey', 'x3lpxg

{'e3zev3', 'qkr7p4', 'lzl7pv', 'vy3jek', 'pr6g65', 'db3yb3', 'e4xgdp', 'goaeqo', 'bgaz6b', '6vz38j', '7drr5d', 'yj6gly', '76b5xe', '94ojmb', 'y695vy', 'x6ppdg', 'lalzyo', 'pd67b7', '33grdz', 'mkzymv', '6643r7', 'zgx4pd', '5kgj9a', 'ag8o69', 'xdv3ge', 'voljjk', 'ev3pdd', '3k37oz', 'apzkz5', 'dvyo7q', 'ke3kmd', 'mygzba', 'l66zdo', '63j8z7', 'zp8kp9', 'emryrd', '8x4jbv', 'e3rlea', '66y4m8', 'xdgd9r', 'j343mm', 'byb8vl', '664pe8', 'data:image', 'rva6ep', '49l5y5', 'dv9o9y', 'md39da', 'ogxady', 'gbzq5z', 'a5o8be', 'j3qm64', 'vdqbae', '94azae', 'j3v9zl', 'oy7elx', 'kegrzd', 'br4438', 'x3lx4g', 'abg5kz', '56pd3d', 'yrp66k', '469q88', 'a56rpk', 'kk84ga', 'xgyp5x', 'x3padg', 'dvymx3', '8g8v84', 'ybxq7j', 'j38v5a', '5zyq6r', 'z6m6p5', '8jdgk5', 'db5jb3', 'ab73pk', 'br4j8l', 've8drr'}
These links are obtained in this iteration: 
78
5558
Scrolled to last server.
Iteration 84: Scraped a total of 85 loaded servers.
{'e3zev3', 'xbdo9x', 'lzl7pv', 'qkr7p4', 'y6dx8k', '8glg4b', 'zzoqb4', '6zq67j', 'kaj

Scrolled to last server.
Iteration 86: Scraped a total of 91 loaded servers.
{'6d6x6j', '94le4b', 'po6dm7', 'qpqbgz', '8glg4b', '384aa8', 'lzxrjv', 'bbbxkp', 'v67jje', 'j3qzxl', 'kem56a', '5eg63d', 'ogze37', '9gvema', 'rvgdm8', 'py7ej7', 'x943xg', '86maqv', '38qbxr', 'wxoope', 'yep4l9', 'vegx9x', 'zovl5p', 'pkpl9m', 'oyv8kv', 'kae6kr', 'emr8ob', 'l8eyao', 'rlo7jx', 'agzm49', 'kkgmm6', 'dv9ry3', 'prxkrm', 'ab6p53', 'jgzx54', '438br5', 'dv64kd', '27avrb', '94lg4b', 'bq55bl', 'data:image', 'zzb6r9', 'mkypyd', 'wkgjv2', '5kde4a', 'ag96rk', 'e56x5d', '8j34q4', 'zgyerp', 'gb3ypx', 'kkzqga', '59kgpa', '38zr88', 'rkaxb8', '668mv6', '7j54kd', 'jkr8xm', 'delzz5', 'evbvy3', '3l6kz8', 'jkldk4', 'agaygz', 'mkxzqa', 'kepjqa', 'wxdg7r', 'lz8x97', 'qk6eg6', '38dkg8', 'lzm86j', '6688g8', 'jk7mol', 'xoy7r5', 'x3rkd5', 'kxl8l6', 'prq875', 'bbla88', 'kk6z3v', 'kkd34r', 'rvmdg8', 'pkbbk8', 'x3a4qg', 'br4j8l'}
These links are obtained in this iteration: 
82
5775
Scrolled to last server.
Iteration 92: Scrape

Scrolled to last server.
Iteration 96: Scraped a total of 95 loaded servers.
{'yjyke5', 'e5blbd', 'j3bpk4', '5emkz6', 'md7v49', 'qklj44', 'zz96qd', 'yjml6j', 'ogxakv', '389day', '36bqqo', '86o8em', '8kmb5b', 'l5qv74', '9qmxyk', 'o4e6vr', '66pprj', '56lmjz', 'd5q9pq', 'dvgrvd', '8jrvdb', 'lz4qeo', '9gvema', '76mzgd', 'goqpjz', 'oorxex', 'z44mb5', 'lzxjjo', 'abe69e', 'zgkpyd', '76k8vv', 'gb7akm', 'x3kpam', 'j34g5k', 'je9ryp', 'mdmdx7', 'emeayp', '4x9qev', 'rdjjm7', 'kevxjd', 'oybvz7', '9gkqak', '369j8y', '27avrb', 'o3rmkj', '3k5r9o', 'emolm3', 'data:image', '43qla8', '56p497', 'yr488m', '2rbm7n', '6b9kgj', 'mkaox7', 'ema69d', 'gk7vgz', 'agxd65', 'qkvq4z', 'kp559v', '5oa8rz', '668mv6', 'gzz4ko', 'rkrpd8', 'qk7m9v', '7d598b', 'pyxxv8', '6688g8', 'a3q4ze', '5ee4od', 'oy3d8y', '56pber', 'x3rkd5', 'ao7xle', 'kaekyd', 'e4ljmm', 'kzd65a', 'qq6vp9', '864yb4'}
These links are obtained in this iteration: 
78
5957
Scrolled to last server.
Iteration 96: Scraped a total of 90 loaded servers.
{'yjyke5

In [15]:
# driver.quit()